# * VINSIGHT : Inflow
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE
    Liveness Detection : Effective 18 Aug 2025 ++ (กระทบ Prepaid เป็นหลัก)

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R00010001CS' --Total Inflow M1
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
        
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
        
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
        , 'B2R020500CS' --Postpaid Inflow M1 B2B
        , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
        , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
        , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
        , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
        , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
        
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
        , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
        , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
        , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

        , 'TB4R001700CS' --TVS Now Inflow M1
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-09-01, 15:48:25

DataFrame: 8748 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_24172\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Inflow M1(All Channel) Yearly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_yearly_df = m1_yearly_df.loc[m1_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_yearly_df['TOTAL'] = np.where(m1_yearly_df['METRIC_CD']=='B0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE'] = np.where(m1_yearly_df['METRIC_CD']=='B1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C'] = np.where(m1_yearly_df['METRIC_CD']=='B2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R010500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B'] = np.where(m1_yearly_df['METRIC_CD']=='B2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R020500CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOL_CONN'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_CONN(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601D1CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602D1CS', m1_yearly_df['P'], 0)
m1_yearly_df['TVS_NOW'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001700CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TVS_CMDU'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001004CS', m1_yearly_df['P'], 0)

# m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
# m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_yearly_df_display = m1_yearly_df.copy()
m1_yearly_df_display['ACTUAL_AS_OF'] = m1_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_yearly_df_display[col] = m1_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-09-01 02:19:35,20250830,0,"4,340,754,174","2,310,738,659","2,030,015,515","3,303,089,886","1,531,665,299","1,778,451,508","638,826,784","498,266,432","140,560,352","214,380,432","2,989,947"


In [5]:
''' Inflow M1(All Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_quarterly_df = m1_quarterly_df.loc[m1_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_quarterly_df['TOTAL'] = np.where(m1_quarterly_df['METRIC_CD']=='B0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE'] = np.where(m1_quarterly_df['METRIC_CD']=='B1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R010500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R020500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOL_CONN'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_CONN(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601D1CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602D1CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TVS_NOW'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001700CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TVS_CMDU'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001004CS', m1_quarterly_df['P'], 0)

# m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
# m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_quarterly_df_display = m1_quarterly_df.copy()
m1_quarterly_df_display['ACTUAL_AS_OF'] = m1_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_quarterly_df_display[col] = m1_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-09-01 02:19:35,20250331,0,"1,778,578,561","918,417,080","860,161,482","1,379,322,046","619,695,416","759,626,629","249,140,023","195,429,182","53,710,841","79,484,928","1,120,189"
1,20252,2025-09-01 02:19:35,20250630,"-219,752,793","1,558,825,768","847,862,075","710,963,693","1,168,938,044","555,464,378","613,473,665","241,679,456","185,021,052","56,658,404","81,798,101","1,176,722"
2,20253,2025-09-01 02:19:35,20250830,"-555,475,924","1,003,349,844","544,459,504","458,890,340","754,829,797","356,505,504","405,351,213","148,007,304","117,816,197","30,191,107","53,097,403","693,036"


In [6]:
''' Inflow M1(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_monthly_df = m1_monthly_df.loc[m1_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_monthly_df['TOTAL'] = np.where(m1_monthly_df['METRIC_CD']=='B0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE'] = np.where(m1_monthly_df['METRIC_CD']=='B1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C'] = np.where(m1_monthly_df['METRIC_CD']=='B2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R010500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B'] = np.where(m1_monthly_df['METRIC_CD']=='B2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R020500CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOL_CONN'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_CONN(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601D1CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602D1CS', m1_monthly_df['P'], 0)
m1_monthly_df['TVS_NOW'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001700CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TVS_CMDU'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001004CS', m1_monthly_df['P'], 0)

# m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
# m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_monthly_df_display = m1_monthly_df.copy()
m1_monthly_df_display['ACTUAL_AS_OF'] = m1_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_monthly_df_display[col] = m1_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-09-01 02:19:35,20250131,0,"639,880,599","319,922,636","319,957,964","508,715,782","225,749,189","282,966,592","79,460,514","61,334,089","18,126,426","25,646,982","382,335"
1,202502,2025-09-01 02:19:35,20250228,"-64,700,198","575,180,401","288,071,249","287,109,152","442,103,679","192,368,192","249,735,487","81,132,607","63,185,463","17,947,144","25,046,745","267,183"
2,202503,2025-09-01 02:19:35,20250331,"-11,662,840","563,517,561","310,423,195","253,094,366","428,502,585","201,578,035","226,924,550","88,546,902","70,909,630","17,637,272","28,791,201","470,671"
3,202504,2025-09-01 02:19:35,20250430,"-29,481,414","534,036,147","295,795,804","238,240,343","395,842,378","191,822,637","204,019,741","87,789,592","69,118,915","18,670,677","26,236,110","441,580"
4,202505,2025-09-01 02:19:35,20250531,"-1,532,940","532,503,207","287,111,076","245,392,131","400,164,161","190,514,414","209,649,746","79,167,109","59,723,743","19,443,366","28,292,314","401,782"
5,202506,2025-09-01 02:19:35,20250630,"-40,216,792","492,286,414","264,955,195","227,331,219","372,931,505","173,127,327","199,804,178","74,722,754","56,178,394","18,544,361","27,269,677","333,361"
6,202507,2025-09-01 02:19:35,20250731,"40,111,551","532,397,966","285,367,456","247,030,509","406,337,686","187,269,889","219,067,796","78,017,438","61,986,278","16,031,160","27,376,187","323,068"
7,202508,2025-09-01 02:19:35,20250830,"-61,446,087","470,951,879","259,092,048","211,859,831","348,492,111","169,235,615","186,283,417","69,989,866","55,829,919","14,159,947","25,721,216","369,968"


In [7]:
''' Inflow M1(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_weekly_df = m1_weekly_df.loc[m1_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_weekly_df['TOTAL'] = np.where(m1_weekly_df['METRIC_CD']=='B0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE'] = np.where(m1_weekly_df['METRIC_CD']=='B1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C'] = np.where(m1_weekly_df['METRIC_CD']=='B2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R010500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B'] = np.where(m1_weekly_df['METRIC_CD']=='B2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R020500CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOL_CONN'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_CONN(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601D1CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602D1CS', m1_weekly_df['P'], 0)
m1_weekly_df['TVS_NOW'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001700CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TVS_CMDU'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001004CS', m1_weekly_df['P'], 0)

# m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
# m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_weekly_df_display = m1_weekly_df.copy()
m1_weekly_df_display['ACTUAL_AS_OF'] = m1_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_weekly_df_display[col] = m1_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025027,2025-09-01 02:19:35,20250706,0,"103,338,663","53,204,818","50,133,845","71,944,801","32,289,169","39,655,631","16,894,941","13,747,291","3,147,650","5,610,610","67,784"
1,2025028,2025-09-01 02:19:35,20250713,"13,214,369","116,553,032","64,376,844","52,176,188","87,406,486","40,690,966","46,715,520","19,607,837","15,957,420","3,650,417","6,456,671","67,178"
2,2025029,2025-09-01 02:19:35,20250720,"4,480,108","121,033,140","65,010,559","56,022,582","92,978,341","41,642,547","51,335,795","18,645,830","14,624,216","4,021,614","6,146,570","63,039"
3,2025030,2025-09-01 02:19:35,20250727,"-1,697,842","119,335,299","62,277,808","57,057,491","95,475,920","43,438,795","52,037,125","14,945,099","11,477,512","3,467,587","5,671,987","64,957"
4,2025031,2025-09-01 02:19:35,20250803,"4,353,136","123,688,435","69,991,445","53,696,989","97,908,047","48,044,764","49,863,283","16,471,920","13,457,381","3,014,539","6,139,326","104,698"
5,2025032,2025-09-01 02:19:35,20250810,"-5,406,053","118,282,382","65,235,885","53,046,497","91,247,222","43,373,491","47,873,731","16,998,002","13,678,053","3,319,949","6,163,082","65,607"
6,2025033,2025-09-01 02:19:35,20250817,"-212,211","118,070,171","64,885,496","53,184,675","92,916,992","45,025,293","47,891,699","15,938,771","12,471,257","3,467,514","5,822,335","95,309"
7,2025034,2025-09-01 02:19:35,20250824,"-14,740,137","103,330,034","59,059,718","44,270,316","79,902,845","39,378,261","40,524,584","15,465,501","11,996,401","3,469,100","5,966,061","90,686"
8,2025035,2025-09-01 02:19:35,20250830,"-23,611,345","79,718,689","40,416,932","39,301,757","45,049,142","22,622,218","29,453,845","13,039,402","10,406,666","2,632,737","5,120,761","73,779"


In [8]:
''' Inflow M1(All Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_daily_df = m1_daily_df.loc[m1_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_daily_df['TOTAL'] = np.where(m1_daily_df['METRIC_CD']=='B0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_T'] = np.where(m1_daily_df['METRIC_CD']=='TB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_D'] = np.where(m1_daily_df['METRIC_CD']=='DB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['PRE'] = np.where(m1_daily_df['METRIC_CD']=='B1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_T'] = np.where(m1_daily_df['METRIC_CD']=='TB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_D'] = np.where(m1_daily_df['METRIC_CD']=='DB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C'] = np.where(m1_daily_df['METRIC_CD']=='B2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R010500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B'] = np.where(m1_daily_df['METRIC_CD']=='B2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R020500CS', m1_daily_df['P'], 0)
m1_daily_df['TOL_CONN'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_CONN(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601D1CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602D1CS', m1_daily_df['P'], 0)
m1_daily_df['TVS_NOW'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001700CS', m1_daily_df['P'], 0)
# m1_daily_df['TVS_CMDU'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001004CS', m1_daily_df['P'], 0)

# m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
# m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_daily_df_display = m1_daily_df.copy()
mod_col_list = m1_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_daily_df_display[col] = m1_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202508,20250831,2025-09-01 02:19:35,"-3,453,064",0,0,0,0,0,0,0,0,0,0,0
1,202508,20250830,2025-09-01 02:19:35,"-8,424,570","3,453,064","3,053,397","399,667",0,0,0,"2,284,538","1,888,033","396,505","831,069",0
2,202508,20250829,2025-09-01 02:19:35,"-3,373,798","11,877,634","3,210,943","8,666,691",0,0,"7,026,921","2,122,425","1,690,048","432,377","861,758","14,551"
3,202508,20250828,2025-09-01 02:19:35,"506,482","15,251,432","8,549,586","6,701,846","12,115,857","6,000,331","6,115,526","1,994,017","1,543,636","450,381","800,516","12,352"
4,202508,20250827,2025-09-01 02:19:35,"-4,850,865","14,744,950","8,772,440","5,972,510","11,186,741","5,777,834","5,408,907","2,023,753","1,588,143","435,610","888,004","16,320"
5,202508,20250826,2025-09-01 02:19:35,"4,800,022","19,595,815","8,297,110","11,298,705","10,553,726","5,274,538","5,279,188","2,347,454","1,895,144","452,310","804,628","14,125"
6,202508,20250825,2025-09-01 02:19:35,"131,280","14,795,794","8,533,456","6,262,338","11,192,818","5,569,515","5,623,303","2,267,216","1,801,662","465,554","934,786","16,431"
7,202508,20250824,2025-09-01 02:19:35,"-481,292","14,664,514","8,421,035","6,243,479","11,388,714","5,470,702","5,918,012","2,396,986","2,073,763","323,223","805,927","18,287"
8,202508,20250823,2025-09-01 02:19:35,"186,044","15,145,806","8,766,528","6,379,277","11,798,948","5,816,585","5,982,362","2,333,335","1,945,640","387,695","929,415","17,270"
9,202508,20250822,2025-09-01 02:19:35,"507,488","14,959,762","8,391,929","6,567,833","11,711,118","5,711,061","6,000,057","2,131,298","1,609,188","522,110","792,269","16,071"


## Overview(Geo) by Period

In [9]:
''' Inflow M1(Geo Channel) Yearly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_yearly_df = m1_geo_yearly_df.loc[m1_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_yearly_df['TOTAL'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R010500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R020500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOL_CONN'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_CONN(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TVS_NOW'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001700CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TVS_CMDU'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001004CG', m1_geo_yearly_df['P'], 0)

# m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
# m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_yearly_df_display = m1_geo_yearly_df.copy()
m1_geo_yearly_df_display['ACTUAL_AS_OF'] = m1_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_yearly_df_display[col] = m1_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-09-01 06:06:49,20250830,0,"3,433,738,064","1,965,897,057","1,467,841,006","2,917,955,313","1,516,829,064","1,406,953,084","308,623,228","251,085,398","57,537,830","168,539,585","2,977,635"


In [10]:
''' Inflow M1(Geo Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_quarterly_df = m1_geo_quarterly_df.loc[m1_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_quarterly_df['TOTAL'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R010500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R020500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOL_CONN'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_CONN(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TVS_NOW'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001700CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TVS_CMDU'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001004CG', m1_geo_quarterly_df['P'], 0)

# m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
# m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_quarterly_df_display = m1_geo_quarterly_df.copy()
m1_geo_quarterly_df_display['ACTUAL_AS_OF'] = m1_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_quarterly_df_display[col] = m1_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-09-01 06:06:49,20250331,0,"1,378,838,395","790,266,697","588,571,699","1,179,496,714","614,011,326","565,485,388","124,890,149","102,959,079","21,931,070","62,397,203","1,111,862"
1,20252,2025-09-01 06:06:49,20250630,"-122,473,299","1,256,365,097","717,939,379","538,425,718","1,064,571,804","549,580,091","514,991,713","115,279,600","93,119,044","22,160,557","64,775,717","1,174,754"
2,20253,2025-09-01 06:06:49,20250830,"-457,830,525","798,534,572","457,690,982","340,843,590","673,886,795","353,237,647","326,475,983","68,453,479","55,007,276","13,446,203","41,366,665","691,020"


In [11]:
''' Inflow M1(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_geo_monthly_df = m1_geo_monthly_df.loc[m1_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_monthly_df['TOTAL'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R010500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R020500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOL_CONN'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_CONN(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TVS_NOW'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001700CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TVS_CMDU'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001004CG', m1_geo_monthly_df['P'], 0)

# m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
# m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_monthly_df_display = m1_geo_monthly_df.copy()
m1_geo_monthly_df_display['ACTUAL_AS_OF'] = m1_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_monthly_df_display[col] = m1_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-09-01 06:06:49,20250131,0,"491,850,599","278,217,598","213,633,001","429,927,804","223,753,635","206,174,169","38,555,159","31,477,331","7,077,828","19,471,134","376,664"
1,202502,2025-09-01 06:06:49,20250228,"-52,938,633","438,911,966","247,017,359","191,894,607","375,063,654","190,565,264","184,498,390","39,986,117","32,971,912","7,014,205","19,961,102","264,749"
2,202503,2025-09-01 06:06:49,20250331,"9,163,864","448,075,830","265,031,740","183,044,091","374,505,256","199,692,427","174,812,829","46,348,874","38,509,836","7,839,038","22,964,967","470,449"
3,202504,2025-09-01 06:06:49,20250430,"-18,662,210","429,413,620","251,562,458","177,851,162","359,407,356","189,643,908","169,763,448","45,183,640","37,483,652","7,699,989","20,901,734","441,246"
4,202505,2025-09-01 06:06:49,20250531,"1,471,543","430,885,163","244,187,397","186,697,766","367,391,197","188,609,826","178,781,371","36,970,047","29,453,492","7,516,555","22,391,575","400,240"
5,202506,2025-09-01 06:06:49,20250630,"-34,818,849","396,066,314","222,189,524","173,876,790","337,773,250","171,326,357","166,446,894","33,125,912","26,181,899","6,944,013","21,482,408","333,269"
6,202507,2025-09-01 06:06:49,20250731,"30,192,530","426,258,844","240,543,317","185,715,526","363,952,007","185,704,939","178,247,068","36,374,590","29,313,716","7,060,874","21,381,863","323,068"
7,202508,2025-09-01 06:06:49,20250830,"-53,983,116","372,275,728","217,147,664","155,128,064","309,934,788","167,532,708","148,228,915","32,078,888","25,693,559","6,385,329","19,984,802","367,952"


In [12]:
''' Inflow M1(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_weekly_df = m1_geo_weekly_df.loc[m1_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_weekly_df['TOTAL'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R010500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R020500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOL_CONN'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_CONN(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TVS_NOW'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001700CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TVS_CMDU'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001004CG', m1_geo_weekly_df['P'], 0)

# m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
# m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_weekly_df_display = m1_geo_weekly_df.copy()
m1_geo_weekly_df_display['ACTUAL_AS_OF'] = m1_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_weekly_df_display[col] = m1_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025027,2025-09-01 06:06:49,20250706,0,"77,338,701","43,167,453","34,171,248","64,385,376","31,990,403","32,394,973","7,803,543","6,148,715","1,654,828","4,451,471","67,784"
1,2025028,2025-09-01 06:06:49,20250713,"15,541,704","92,880,405","52,870,425","40,009,980","78,320,338","40,207,114","38,113,223","8,788,350","6,986,958","1,801,392","5,112,148","67,178"
2,2025029,2025-09-01 06:06:49,20250720,"5,328,755","98,209,160","54,599,354","43,609,806","83,111,115","41,296,726","41,814,389","9,074,014","7,369,896","1,704,117","4,696,945","63,039"
3,2025030,2025-09-01 06:06:49,20250727,"-868,495","97,340,665","54,029,083","43,311,583","85,182,112","43,163,979","42,018,134","6,844,504","5,624,473","1,220,031","4,372,579","64,957"
4,2025031,2025-09-01 06:06:49,20250803,"4,067,029","101,407,695","59,897,364","41,510,330","87,560,627","47,729,182","39,831,445","7,681,536","6,164,226","1,517,310","4,788,641","104,698"
5,2025032,2025-09-01 06:06:49,20250810,"-6,900,305","94,507,390","54,958,712","39,548,678","80,814,821","42,996,162","37,818,659","7,974,454","6,346,962","1,627,492","4,770,202","65,607"
6,2025033,2025-09-01 06:06:49,20250817,"692,286","95,199,676","55,123,413","40,076,263","82,998,497","44,524,837","38,473,661","6,954,870","5,478,650","1,476,220","4,479,683","93,814"
7,2025034,2025-09-01 06:06:49,20250824,"-11,890,292","83,309,384","50,128,398","33,180,986","70,769,232","38,963,325","31,805,907","7,045,084","5,717,700","1,327,384","4,554,619","90,444"
8,2025035,2025-09-01 06:06:49,20250830,"-24,967,889","58,341,495","32,916,779","25,424,717","40,744,677","22,365,919","24,205,593","6,287,124","5,169,695","1,117,429","4,140,377","73,500"


In [13]:
''' Inflow M1(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_daily_df['TOTAL'] = np.where(m1_geo_daily_df['METRIC_CD']=='B0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE'] = np.where(m1_geo_daily_df['METRIC_CD']=='B1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R010500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R020500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL_CONN'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_CONN(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TVS_NOW'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001700CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TVS_CMDU'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001004CG', m1_geo_daily_df['P'], 0)

# m1_geo_daily_df = m1_geo_daily_df.groupby('TM_KEY_MTH', 'TM_KEY_DAY').agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
# m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_daily_df = m1_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_daily_df_display = m1_geo_daily_df.copy()
mod_col_list = m1_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_daily_df_display[col] = m1_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202508,20250831,2025-09-01 06:06:49,"-2,024,276",0,0,0,0,0,0,0,0,0,0,0
1,202508,20250830,2025-09-01 06:06:49,"-5,936,636","2,024,276","1,818,232","206,044",0,0,0,"1,102,271","899,389","202,882","617,309",0
2,202508,20250829,2025-09-01 06:06:49,"-4,788,078","7,960,913","1,932,513","6,028,400",0,0,"5,826,835","1,037,954","856,363","181,591","709,508","14,551"
3,202508,20250828,2025-09-01 06:06:49,"660,703","12,748,991","7,513,814","5,235,176","10,975,108","5,925,351","5,049,757","1,002,644","829,834","172,810","653,513","12,352"
4,202508,20250827,2025-09-01 06:06:49,"687,541","12,088,288","7,484,748","4,603,540","10,120,401","5,707,408","4,412,993","1,016,229","836,957","179,272","729,093","16,320"
5,202508,20250826,2025-09-01 06:06:49,"-717,534","11,400,747","6,887,119","4,513,627","9,538,440","5,222,636","4,315,804","1,091,660","899,346","192,314","649,657","14,125"
6,202508,20250825,2025-09-01 06:06:49,"194,457","12,118,281","7,280,352","4,837,929","10,110,728","5,510,525","4,600,203","1,036,366","847,806","188,560","781,297","16,151"
7,202508,20250824,2025-09-01 06:06:49,"-168,650","11,923,824","6,914,485","5,009,340","10,184,812","5,398,919","4,785,893","1,114,467","893,264","221,203","576,958","18,287"
8,202508,20250823,2025-09-01 06:06:49,"-32,939","12,092,474","7,283,320","4,809,154","10,361,586","5,752,285","4,609,301","1,032,091","835,038","197,053","653,177","17,270"
9,202508,20250822,2025-09-01 06:06:49,"439,072","12,125,413","7,231,785","4,893,628","10,376,880","5,655,318","4,721,562","970,117","808,634","161,483","624,103","15,829"


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202508


### All Service

In [16]:
''' All Service (All Channel) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_NAME'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,All Services,B0R00010001CS,Total Inflow M1,2025-09-01 02:19:35,"470,951,879","470,951,879",0,0,0
1,202508,All Services,DB0R00010001CS,Total Inflow M1 : DTAC,2025-09-01 02:19:35,"211,859,831","211,859,831",0,0,0
2,202508,All Services,TB0R00010001CS,Total Inflow M1 : TRUE,2025-09-01 02:19:35,"259,092,048","259,092,048",0,0,0


In [17]:
''' All Service (Geo Channel) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_NAME'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,All Services,B0R00010001CG,Total Inflow M1 - GEO Channel,2025-09-01 06:06:49,0,"372,275,728","326,975,491","326,983,521","320,943,336"
1,202508,All Services,DB0R00010001CG,Total Inflow M1 : DTAC - GEO Channel,2025-09-01 06:06:49,0,"155,128,064","154,769,119","154,769,119","151,678,323"
2,202508,All Services,TB0R00010001CG,Total Inflow M1 : TRUE - GEO Channel,2025-09-01 06:06:49,0,"217,147,664","172,206,372","172,214,402","169,265,013"


### Prepaid

In [18]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Prepaid,B1R000900CS,Prepaid Inflow M1,2025-09-01 02:19:35,"348,492,111","348,492,111",0,0,0
1,202508,Prepaid,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-09-01 02:19:35,"186,283,417","186,283,417",0,0,0
2,202508,Prepaid,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-09-01 02:19:35,"169,235,615","169,235,615",0,0,0


In [19]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Prepaid,B1R000900CG,Prepaid Inflow M1 - GEO Channel,2025-09-01 06:06:49,0,"309,934,788","264,683,232","264,683,232","259,031,171"
1,202508,Prepaid,DB1R000900CG,Prepaid Inflow M1 : DTAC - GEO Channel,2025-09-01 06:06:49,0,"148,228,915","147,877,652","147,877,652","144,786,856"
2,202508,Prepaid,TB1R000900CG,Prepaid Inflow M1 : TMH - GEO Channel,2025-09-01 06:06:49,0,"167,532,708","122,595,860","122,595,860","119,900,790"


### Postpaid

In [20]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2R010500CS,Postpaid Inflow M1 B2C,2025-09-01 02:19:35,"69,989,866","69,989,866",0,0,0
1,202508,Postpaid,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-09-01 02:19:35,"14,159,947","14,159,947",0,0,0
2,202508,Postpaid,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-09-01 02:19:35,"55,829,919","55,829,919",0,0,0


In [21]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2R010500CG,Postpaid Inflow M1 B2C - GEO Channel,2025-09-01 06:06:49,0,"32,078,888","32,069,111","31,599,771","30,828,758"
1,202508,Postpaid,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,2025-09-01 06:06:49,0,"6,385,329","6,377,647","6,377,647","6,377,647"
2,202508,Postpaid,TB2R010500CG,Postpaid Inflow M1 B2C : TMH - GEO Channel,2025-09-01 06:06:49,0,"25,693,559","25,691,464","25,691,464","25,691,464"


In [22]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2R020500CS,Postpaid Inflow M1 B2B,2025-09-01 02:19:35,"17,474,840","17,474,840",0,0,0
1,202508,Postpaid,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-09-01 02:19:35,"11,416,467","11,416,467",0,0,0
2,202508,Postpaid,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-09-01 02:19:35,"6,058,373","6,058,373",0,0,0


In [23]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2R020500CG,Postpaid Inflow M1 B2B - GEO Channel,2025-09-01 06:06:49,0,"3,981,606","3,408,108","2,812,431","3,030,348"
1,202508,Postpaid,DB2R020500CG,Postpaid Inflow M1 B2B : DTAC - GEO Channel,2025-09-01 06:06:49,0,"513,820","513,820","513,820","513,820"
2,202508,Postpaid,TB2R020500CG,Postpaid Inflow M1 B2B : TMH - GEO Channel,2025-09-01 06:06:49,0,"3,467,786","3,467,387","3,475,417","3,476,016"


### TOL

In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-09-01 02:19:35,"25,721,216","25,721,216",0,0,0
1,202508,TOL,TB3R000601D1CS,TOL Inflow M1 Connected : Consumer (Install Lo...,2025-09-01 02:19:35,"25,721,216","25,721,216",0,0,0


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3R000601CG,TOL Inflow M1 Connected : Consumer - GEO Channel,2025-09-01 06:06:49,0,"19,984,802","19,982,852","19,982,852","19,727,933"
1,202508,TOL,TB3R000601D1CG,TOL Inflow M1 Connected : Consumer - GEO Chann...,2025-09-01 06:06:49,0,"19,984,802","19,984,802","19,984,802","19,984,802"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-09-01 02:19:35,"1,613,500","1,613,500",0,0,0
1,202508,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-09-01 02:19:35,"1,613,500","1,613,500",0,0,0


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-09-01 02:19:35,"1,613,500","1,613,500",0,0,0
1,202508,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-09-01 02:19:35,"1,613,500","1,613,500",0,0,0


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TVS,TB4R001700CS,TVS Now Inflow M1,2025-09-01 02:19:35,"369,968","369,968",0,0,0
1,202508,TVS,TB4R001700CG,TVS Now Inflow M1 - GEO Channel,2025-09-01 06:06:49,0,"367,952","367,952","367,952","367,952"


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TVS,TB4R001004CS,TVS CMDU Inflow M1,2025-09-01 02:19:35,"263,458","263,458",0,0,0
1,202508,TVS,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,2025-09-01 06:06:49,0,0,0,0,0


## ** Special Metric (All & Geo)

### Inflow (All Channel)
    B0R00010001CS   Total Inflow M1
    TB0R00010001CS  Total Inflow M1 : TRUE
    DB0R00010001CS  Total Inflow M1 : DTAC

In [30]:
''' B0R00010001CS : Total Inflow M1 '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'DB2R010500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_df = total_m1_df.loc[total_m1_df['METRIC_CD'].isin(v_metric_list)]

total_m1_df['TOTAL'] = np.where(total_m1_df['METRIC_CD']=='B0R00010001CS', total_m1_df['P'], 0)

total_m1_df['PRE_T'] = np.where(total_m1_df['METRIC_CD']=='TB1R000900CS', total_m1_df['P'], 0)
total_m1_df['PRE_D'] = np.where(total_m1_df['METRIC_CD']=='DB1R000900CS', total_m1_df['P'], 0)

total_m1_df['POST_B2C_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2C_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R020500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R020500CS', total_m1_df['P'], 0)

total_m1_df['TOL_CON'] = np.where(total_m1_df['METRIC_CD']=='TB3R000601CS', total_m1_df['P'], 0)
total_m1_df['TOL_DSV'] = np.where(total_m1_df['METRIC_CD']=='TB3R000602CS', total_m1_df['P'], 0)

total_m1_df['TVS_NOW'] = np.where(total_m1_df['METRIC_CD']=='TB4R001700CS', total_m1_df['P'], 0)
total_m1_df['TVS_CMDU'] = np.where(total_m1_df['METRIC_CD']=='TB4R001004CS', total_m1_df['P'], 0)

total_m1_df['CHK_SUM'] = np.where(total_m1_df['METRIC_CD'].isin(['TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'DB2R010500CS', 'TB2R020500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_df['P'], 0)

total_m1_df = total_m1_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_df['CHK_DIFF'] = total_m1_df['TOTAL'] - total_m1_df['CHK_SUM']
total_m1_df = total_m1_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_df = total_m1_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_df[col] = total_m1_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-09-01 02:19:35,0,"639,880,599","639,880,599","225,749,189","282,966,592","61,334,089","18,126,426","4,946,013","18,864,945","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-09-01 02:19:35,0,"575,180,401","575,180,401","192,368,192","249,735,487","63,185,463","17,947,144","4,637,783","19,426,521","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-09-01 02:19:35,0,"563,517,561","563,517,561","201,578,035","226,924,550","70,909,630","17,637,272","5,801,792","8,532,545","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-09-01 02:19:35,-0,"534,036,147","534,036,147","191,822,637","204,019,741","69,118,915","18,670,677","5,402,830","15,549,925","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-09-01 02:19:35,0,"532,503,207","532,503,207","190,514,414","209,649,746","59,723,743","19,443,366","6,239,659","16,299,019","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-09-01 02:19:35,0,"492,286,414","492,286,414","173,127,327","199,804,178","56,178,394","18,544,361","6,226,938","8,982,680","27,269,677","1,643,369","333,361","176,129"
6,202507,2025-09-01 02:19:35,0,"532,397,966","532,397,966","187,269,889","219,067,796","61,986,278","16,031,160","6,758,180","11,931,553","27,376,187","1,528,784","323,068","125,070"
7,202508,2025-09-01 02:19:35,-0,"470,951,879","470,951,879","169,235,615","186,283,417","55,829,919","14,159,947","6,058,373","11,416,467","25,721,216","1,613,500","369,968","263,458"


In [31]:
''' TB0R00010001CS : Total Inflow M1 : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'TB0R00010001CS' #Total Inflow M1 : TRUE

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_df = total_m1_true_df.loc[total_m1_true_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_df['TOTAL_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB0R00010001CS', total_m1_true_df['P'], 0)

total_m1_true_df['PRE_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB1R000900CS', total_m1_true_df['P'], 0)

total_m1_true_df['POST_B2C_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R010500CS', total_m1_true_df['P'], 0)
total_m1_true_df['POST_B2B_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R020500CS', total_m1_true_df['P'], 0)

total_m1_true_df['TOL_CON'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000601CS', total_m1_true_df['P'], 0)
total_m1_true_df['TOL_DSV'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000602CS', total_m1_true_df['P'], 0)

total_m1_true_df['TVS_NOW'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001700CS', total_m1_true_df['P'], 0)
total_m1_true_df['TVS_CMDU'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001004CS', total_m1_true_df['P'], 0)

total_m1_true_df['CHK_SUM'] = np.where(total_m1_true_df['METRIC_CD'].isin(['TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_true_df['P'], 0)

total_m1_true_df = total_m1_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_df['CHK_DIFF'] = total_m1_true_df['TOTAL_T'] - total_m1_true_df['CHK_SUM']
total_m1_true_df = total_m1_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_df = total_m1_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_df[col] = total_m1_true_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-09-01 02:19:35,0,"319,922,636","319,922,636","225,749,189","61,334,089","4,946,013","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-09-01 02:19:35,0,"288,071,249","288,071,249","192,368,192","63,185,463","4,637,783","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-09-01 02:19:35,0,"310,423,195","310,423,195","201,578,035","70,909,630","5,801,792","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-09-01 02:19:35,0,"295,795,804","295,795,804","191,822,637","69,118,915","5,402,830","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-09-01 02:19:35,0,"287,111,076","287,111,076","190,514,414","59,723,743","6,239,659","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-09-01 02:19:35,0,"264,955,195","264,955,195","173,127,327","56,178,394","6,226,938","27,269,677","1,643,369","333,361","176,129"
6,202507,2025-09-01 02:19:35,0,"285,367,456","285,367,456","187,269,889","61,986,278","6,758,180","27,376,187","1,528,784","323,068","125,070"
7,202508,2025-09-01 02:19:35,0,"259,092,048","259,092,048","169,235,615","55,829,919","6,058,373","25,721,216","1,613,500","369,968","263,458"


In [32]:
''' DB0R00010001CS : Total Inflow M1 : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS')

v_metric_list = [
    'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    ]

total_m1_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_df = total_m1_dtac_df.loc[total_m1_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_df['TOTAL_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB0R00010001CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['PRE_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB1R000900CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['POST_B2C_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R010500CS', total_m1_dtac_df['P'], 0)
total_m1_dtac_df['POST_B2B_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R020500CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['CHK_SUM'] = np.where(total_m1_dtac_df['METRIC_CD'].isin(['DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS']), total_m1_dtac_df['P'], 0)

total_m1_dtac_df = total_m1_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_df['CHK_DIFF'] = total_m1_dtac_df['TOTAL_D'] - total_m1_dtac_df['CHK_SUM']
total_m1_dtac_df = total_m1_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_df = total_m1_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_df[col] = total_m1_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-09-01 02:19:35,0,"319,957,964","319,957,964","282,966,592","18,126,426","18,864,945"
1,202502,2025-09-01 02:19:35,0,"287,109,152","287,109,152","249,735,487","17,947,144","19,426,521"
2,202503,2025-09-01 02:19:35,0,"253,094,366","253,094,366","226,924,550","17,637,272","8,532,545"
3,202504,2025-09-01 02:19:35,0,"238,240,343","238,240,343","204,019,741","18,670,677","15,549,925"
4,202505,2025-09-01 02:19:35,0,"245,392,131","245,392,131","209,649,746","19,443,366","16,299,019"
5,202506,2025-09-01 02:19:35,0,"227,331,219","227,331,219","199,804,178","18,544,361","8,982,680"
6,202507,2025-09-01 02:19:35,0,"247,030,509","247,030,509","219,067,796","16,031,160","11,931,553"
7,202508,2025-09-01 02:19:35,0,"211,859,831","211,859,831","186,283,417","14,159,947","11,416,467"


### Inflow (Geo Channel)
    B0R00010001CG   Total Inflow M1 - GEO Channel
    TB0R00010001CG  Total Inflow M1 : TRUE - GEO Channel
    DB0R00010001CG  Total Inflow M1 : DTAC - GEO Channel

In [33]:
''' B0R00010001CG : Total Inflow M1 - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_geo_df = total_m1_geo_df.loc[total_m1_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_geo_df['TOTAL'] = np.where(total_m1_geo_df['METRIC_CD']=='B0R00010001CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['PRE_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['PRE_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['POST_B2C_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2C_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TOL_CON'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TOL_DSV'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TVS_NOW'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TVS_CMDU'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['CHK_SUM'] = np.where(total_m1_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_geo_df['P'], 0)

total_m1_geo_df = total_m1_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_geo_df['CHK_DIFF'] = total_m1_geo_df['TOTAL'] - total_m1_geo_df['CHK_SUM']
total_m1_geo_df = total_m1_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_geo_df = total_m1_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_geo_df[col] = total_m1_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-09-01 06:06:49,0,"491,850,599","491,850,599","223,753,635","206,174,169","31,477,331","7,077,828","2,357,448","381,005","19,471,134","397,930","376,664","383,456"
1,202502,2025-09-01 06:06:49,-0,"438,911,966","438,911,966","190,565,264","184,498,390","32,971,912","7,014,205","2,468,889","382,012","19,961,102","423,582","264,749","361,861"
2,202503,2025-09-01 06:06:49,0,"448,075,830","448,075,830","199,692,427","174,812,829","38,509,836","7,839,038","3,206,656","392,224","22,964,967","133,660","470,449","53,744"
3,202504,2025-09-01 06:06:49,-0,"429,413,620","429,413,620","189,643,908","169,763,448","37,483,652","7,699,989","2,886,311","387,725","20,901,734","134,885","441,246","70,723"
4,202505,2025-09-01 06:06:49,0,"430,885,163","430,885,163","188,609,826","178,781,371","29,453,492","7,516,555","3,110,301","399,840","22,391,575","221,363","400,240",600
5,202506,2025-09-01 06:06:49,0,"396,066,314","396,066,314","171,326,357","166,446,894","26,181,899","6,944,013","2,765,249","485,883","21,482,408","94,361","333,269","5,981"
6,202507,2025-09-01 06:06:49,-0,"426,258,844","426,258,844","185,704,939","178,247,068","29,313,716","7,060,874","3,661,105","407,584","21,381,863","149,841","323,068","8,785"
7,202508,2025-09-01 06:06:49,-0,"372,275,728","372,275,728","167,532,708","148,228,915","25,693,559","6,385,329","3,467,786","513,820","19,984,802","100,857","367,952",0


In [34]:
''' TB0R00010001CG : Total Inflow M1 : TRUE - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'TB0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_true_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_geo_df = total_m1_true_geo_df.loc[total_m1_true_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_geo_df['TOTAL_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB0R00010001CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['PRE_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['POST_B2C_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['POST_B2B_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TOL_CON'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TOL_DSV'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TVS_NOW'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TVS_CMDU'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['CHK_SUM'] = np.where(total_m1_true_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df = total_m1_true_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_geo_df['CHK_DIFF'] = total_m1_true_geo_df['TOTAL_T'] - total_m1_true_geo_df['CHK_SUM']
total_m1_true_geo_df = total_m1_true_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_geo_df = total_m1_true_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_geo_df[col] = total_m1_true_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-09-01 06:06:49,0,"278,217,598","278,217,598","223,753,635","31,477,331","2,357,448","19,471,134","397,930","376,664","383,456"
1,202502,2025-09-01 06:06:49,0,"247,017,359","247,017,359","190,565,264","32,971,912","2,468,889","19,961,102","423,582","264,749","361,861"
2,202503,2025-09-01 06:06:49,0,"265,031,740","265,031,740","199,692,427","38,509,836","3,206,656","22,964,967","133,660","470,449","53,744"
3,202504,2025-09-01 06:06:49,-0,"251,562,458","251,562,458","189,643,908","37,483,652","2,886,311","20,901,734","134,885","441,246","70,723"
4,202505,2025-09-01 06:06:49,0,"244,187,397","244,187,397","188,609,826","29,453,492","3,110,301","22,391,575","221,363","400,240",600
5,202506,2025-09-01 06:06:49,0,"222,189,524","222,189,524","171,326,357","26,181,899","2,765,249","21,482,408","94,361","333,269","5,981"
6,202507,2025-09-01 06:06:49,0,"240,543,317","240,543,317","185,704,939","29,313,716","3,661,105","21,381,863","149,841","323,068","8,785"
7,202508,2025-09-01 06:06:49,0,"217,147,664","217,147,664","167,532,708","25,693,559","3,467,786","19,984,802","100,857","367,952",0


In [35]:
''' DB0R00010001CG : Total Inflow M1 : DTAC - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG')

v_metric_list = [
    'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel
    
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    ]

total_m1_dtac_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_geo_df = total_m1_dtac_geo_df.loc[total_m1_dtac_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_geo_df['TOTAL_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB0R00010001CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['PRE_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['POST_B2C_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_dtac_geo_df['P'], 0)
total_m1_dtac_geo_df['POST_B2B_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['CHK_SUM'] = np.where(total_m1_dtac_geo_df['METRIC_CD'].isin(['DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG']), total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df = total_m1_dtac_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_geo_df['CHK_DIFF'] = total_m1_dtac_geo_df['TOTAL_D'] - total_m1_dtac_geo_df['CHK_SUM']
total_m1_dtac_geo_df = total_m1_dtac_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_geo_df = total_m1_dtac_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_geo_df[col] = total_m1_dtac_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-09-01 06:06:49,-0,"213,633,001","213,633,001","206,174,169","7,077,828","381,005"
1,202502,2025-09-01 06:06:49,0,"191,894,607","191,894,607","184,498,390","7,014,205","382,012"
2,202503,2025-09-01 06:06:49,0,"183,044,091","183,044,091","174,812,829","7,839,038","392,224"
3,202504,2025-09-01 06:06:49,0,"177,851,162","177,851,162","169,763,448","7,699,989","387,725"
4,202505,2025-09-01 06:06:49,0,"186,697,766","186,697,766","178,781,371","7,516,555","399,840"
5,202506,2025-09-01 06:06:49,0,"173,876,790","173,876,790","166,446,894","6,944,013","485,883"
6,202507,2025-09-01 06:06:49,0,"185,715,526","185,715,526","178,247,068","7,060,874","407,584"
7,202508,2025-09-01 06:06:49,0,"155,128,064","155,128,064","148,228,915","6,385,329","513,820"
